# DS3 SQL Workshop

### Monday, January 28, 2023
### Baraa Zekeria and Ojas Vashishtha

## Setup

In [1]:
# upgrade pip
!{sys.executable} -m pip install --upgrade pip

# install external libraries
import sys
!{sys.executable} -m pip install pandas # data manipulation & analysis
!{sys.executable} -m pip install sqlite3 # SQL

# import libraries
import pandas as pd
import sqlite3

# notebook configurations
pd.options.display.max_colwidth = 1000

import warnings
warnings.filterwarnings("ignore")

print("\n**ALL LIBRARIES IMPORTED SUCCESSFULLY**")

zsh:1: parse error near `-m'
ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3

**ALL LIBRARIES IMPORTED SUCCESSFULLY**


## Connect to SQLite Database

### Step 1: Import Library
[sqlite3](https://docs.python.org/3/library/sqlite3.html)

In [2]:
import sqlite3

### Step 2: Create a connection

In [3]:
connection = sqlite3.connect("ds3_key.db")


### Step 3: Create a cursor object

In [4]:
crsr = connection.cursor()

### Step 4: Convert pandas DataFrame() to a SQL Table

In [5]:
independents = pd.read_csv("data/Independents100.csv")
independents = independents.assign(sales = independents["sales"].apply(int))
ratings = pd.read_csv("data/independents_ratings.csv")

In [6]:
independents.to_sql("independents", con = connection)
ratings.to_sql("ratings", con = connection)

#### Datasets info
- ```independents```: 100 highest-grossing independent restaurants in the US
    - Sorted by ```sales``` in *descending* order
- ```ratings```: Yelp ratings from March 2021 for restaurants in ```independents```

### Step 5: Create queries!

In [7]:
pd.read_sql('''SELECT * 
            FROM independents''', connection)

,index,rank,restaurant,sales,avg_check,city,state,meals_served
0,0,1,Carmine's (Times Square),39080335,40,New York,N.Y.,469803.0
1,1,2,The Boathouse Orlando,35218364,43,Orlando,Fla.,820819.0
2,2,3,Old Ebbitt Grill,29104017,33,Washington,D.C.,892830.0
3,3,4,LAVO Italian Restaurant & Nightclub,26916180,90,New York,N.Y.,198500.0
4,4,5,Bryant Park Grill & Cafe,26900000,62,New York,N.Y.,403000.0
...,...,...,...,...,...,...,...,...
95,95,96,George's at the Cove,12194000,80,La Jolla,Calif.,250000.0
96,96,97,Le Coucou,12187523,95,New York,N.Y.,87070.0
97,97,98,Mi Vida,12032014,38,Washington,D.C.,226226.0
98,98,99,Upland,11965564,52,New York,N.Y.,171825.0


In [8]:
pd.read_sql('''SELECT * 
            FROM ratings''', connection)

,index,restaurant,city,rating
0,0,15th Street Fisheries,Fort Lauderdale,3.5
1,1,Abe & Louie's,Boston,4.0
2,2,Acme Feed & Seed,Nashville,4.0
3,3,Angus Barn,Raleigh,4.0
4,4,Atlanta Fish Market,Atlanta,3.5
...,...,...,...,...
86,86,Top of the World at the STRAT,Las Vegas,4.5
87,87,Upland,New York,4.0
88,88,Virgil's Real Barbecue,New York,3.5
89,89,Virgil's Real Barbecue,Las Vegas,4.0


## Basic Commands

| Keyword | Description
| :- | :- 
| **SELECT**|Selects data from a database
| **AS**| Renames a columns or table with an alias
| **FROM**| Specifies which table to select or delete data from
| **WHERE**| Filters a result set to include only records that fulfill a specified condition
| **JOIN**| Joins tables (right, left, inner, outer, etc.)
| **GROUP BY**| Groups the result set (used with aggregate functions: ```COUNT```, ```MAX```, ```MIN```, ```SUM```, ```AVG```)
| **HAVING**| Used instead of ```WHERE``` with aggreagte functions
| **ORDER BY**| Sorts the result set in ascending or descending order

## Ex 1: Write a query to output all of the restaurants in California and New York

|    |   rank | restaurant                                  |    sales |   avg_check | city           | state   |   meals_served |
|---:|-------:|:--------------------------------------------|---------:|------------:|:---------------|:--------|---------------:|
|  0 |      1 | Carmine's (Times Square)                    | 39080335 |          40 | New York       | N.Y.    |         469803 |
|  3 |      4 | LAVO Italian Restaurant & Nightclub         | 26916180 |          90 | New York       | N.Y.    |         198500 |
|  4 |      5 | Bryant Park Grill & Cafe                    | 26900000 |          62 | New York       | N.Y.    |         403000 |
|  8 |      9 | Balthazar                                   | 24547800 |          87 | New York       | N.Y.    |         519000 |
|  9 |     10 | Smith & Wollensky                           | 24501000 |         107 | New York       | N.Y.    |         257364 |
| ... |     ... | ...                     | ... |          ... | ...       | ...    |         ... |
| 92 |     93 | Virgil's Real Barbecue                      | 12245998 |          31 | New York       | N.Y.    |         251800 |
| 94 |     95 | Franciscan Crab Restaurant                  | 12218147 |          59 | San Francisco  | Calif.  |         240000 |
| 95 |     96 | George's at the Cove                        | 12194000 |          80 | La Jolla       | Calif.  |         250000 |
| 96 |     97 | Le Coucou                                   | 12187523 |          95 | New York       | N.Y.    |          87070 |
| 98 |     99 | Upland                                      | 11965564 |          52 | New York       | N.Y.    |         171825 |

33 rows x 7 columns

In [9]:
pd_ca_ny = independents[independents["state"].isin(["Calif.", "N.Y."])]
pd_ca_ny

,rank,restaurant,sales,avg_check,city,state,meals_served
0,1,Carmine's (Times Square),39080335,40,New York,N.Y.,469803.0
3,4,LAVO Italian Restaurant & Nightclub,26916180,90,New York,N.Y.,198500.0
4,5,Bryant Park Grill & Cafe,26900000,62,New York,N.Y.,403000.0
8,9,Balthazar,24547800,87,New York,N.Y.,519000.0
9,10,Smith & Wollensky,24501000,107,New York,N.Y.,257364.0
13,14,Junior's (Times Square),23640652,23,New York,N.Y.,914500.0
19,20,Buddakan,22395577,67,New York,N.Y.,221059.0
30,31,Quality Meats,18678000,115,New York,N.Y.,165148.0
31,32,Del Posto,18625033,194,New York,N.Y.,95000.0
33,34,Bottega Louie,18521000,36,Los Angeles,Calif.,322562.0


In [10]:
sql_ca_ny = pd.read_sql('''SELECT * 
                        FROM independents 
                        WHERE state IN ("Calif.", "N.Y.")''', connection)
sql_ca_ny

,index,rank,restaurant,sales,avg_check,city,state,meals_served
0,0,1,Carmine's (Times Square),39080335,40,New York,N.Y.,469803.0
1,3,4,LAVO Italian Restaurant & Nightclub,26916180,90,New York,N.Y.,198500.0
2,4,5,Bryant Park Grill & Cafe,26900000,62,New York,N.Y.,403000.0
3,8,9,Balthazar,24547800,87,New York,N.Y.,519000.0
4,9,10,Smith & Wollensky,24501000,107,New York,N.Y.,257364.0
5,13,14,Junior's (Times Square),23640652,23,New York,N.Y.,914500.0
6,19,20,Buddakan,22395577,67,New York,N.Y.,221059.0
7,30,31,Quality Meats,18678000,115,New York,N.Y.,165148.0
8,31,32,Del Posto,18625033,194,New York,N.Y.,95000.0
9,33,34,Bottega Louie,18521000,36,Los Angeles,Calif.,322562.0


## Q1: Write a query to output the name of restaurants in California that has an average check less than 50. 

|    | restaurant                    |
|---:|:------------------------------|
| 33 | Bottega Louie                 |
| 39 | Original Joe's Westlake       |
| 46 | Harris Ranch Inn & Restaurant |
| 70 | Cliff House                   |
| 78 | Paradise Cove Beach Cafe      |
| 80 | Original Joe's                |

6 rows x 1 column

In [11]:
pd_ca_50 = independents[(independents["state"] == "Calif.") & (independents["avg_check"] < 50)][["restaurant"]]
pd_ca_50

,restaurant
33,Bottega Louie
39,Original Joe's Westlake
46,Harris Ranch Inn & Restaurant
70,Cliff House
78,Paradise Cove Beach Cafe
80,Original Joe's


In [12]:
sql_ca_ny = pd.read_sql('''SELECT restaurant
                        FROM independents 
                        WHERE state = "Calif." AND avg_check < 50''', connection)
sql_ca_ny

,restaurant
0,Bottega Louie
1,Original Joe's Westlake
2,Harris Ranch Inn & Restaurant
3,Cliff House
4,Paradise Cove Beach Cafe
5,Original Joe's


## Q2: Write a query to merge restaurants and rankings. Assign to ```sql_merge```.

|    |   rank | restaurant                                  |    sales |   avg_check | city            | state   |   meals_served |   rating |
|---:|-------:|:--------------------------------------------|---------:|------------:|:----------------|:--------|---------------:|---------:|
|  0 |      1 | Carmine's (Times Square)                    | 39080335 |          40 | New York        | N.Y.    |         469803 |      4   |
|  1 |      2 | The Boathouse Orlando                       | 35218364 |          43 | Orlando         | Fla.    |         820819 |      4   |
|  2 |      4 | LAVO Italian Restaurant & Nightclub         | 26916180 |          90 | New York        | N.Y.    |         198500 |      2.5 |
|  3 |      5 | Bryant Park Grill & Cafe                    | 26900000 |          62 | New York        | N.Y.    |         403000 |      3.5 |
|  4 |      6 | Gibsons Bar & Steakhouse                    | 25409952 |          80 | Chicago         | Ill.    |         348567 |      4   |
|  ... |      ... | ...              | ... |         ... | ...       | ...    |         ... |      ... |
| 86 |     95 | Franciscan Crab Restaurant                  | 12218147 |          59 | San Francisco   | Calif.  |         240000 |      3.5 |
| 87 |     96 | George's at the Cove                        | 12194000 |          80 | La Jolla        | Calif.  |         250000 |      4   |
| 88 |     97 | Le Coucou                                   | 12187523 |          95 | New York        | N.Y.    |          87070 |      4   |
| 89 |     99 | Upland                                      | 11965564 |          52 | New York        | N.Y.    |         171825 |      4   |
| 90 |    100 | Virgil's Real Barbecue                      | 11391678 |          27 | Las Vegas       | Nev.    |         208276 |      4   |

91 rows x 8 columns

In [13]:
sql_merge = pd.read_sql('''SELECT independents.rank, independents.restaurant, independents.sales, 
                        independents.avg_check, independents.city, independents.state,
                        independents.meals_served, ratings.rating
                        
                        FROM independents 
                        
                        INNER JOIN ratings ON independents.restaurant  = ratings.restaurant 
                                                          AND 
                                           independents.city  = ratings.city;''', connection)
sql_merge

,rank,restaurant,sales,avg_check,city,state,meals_served,rating
0,1,Carmine's (Times Square),39080335,40,New York,N.Y.,469803.0,4.0
1,2,The Boathouse Orlando,35218364,43,Orlando,Fla.,820819.0,4.0
2,4,LAVO Italian Restaurant & Nightclub,26916180,90,New York,N.Y.,198500.0,2.5
3,5,Bryant Park Grill & Cafe,26900000,62,New York,N.Y.,403000.0,3.5
4,6,Gibsons Bar & Steakhouse,25409952,80,Chicago,Ill.,348567.0,4.0
...,...,...,...,...,...,...,...,...
86,95,Franciscan Crab Restaurant,12218147,59,San Francisco,Calif.,240000.0,3.5
87,96,George's at the Cove,12194000,80,La Jolla,Calif.,250000.0,4.0
88,97,Le Coucou,12187523,95,New York,N.Y.,87070.0,4.0
89,99,Upland,11965564,52,New York,N.Y.,171825.0,4.0


Let's save ```sql_merge``` as a SQL table for easy access.

If you accidently delete the code below, please run the following in a new cell to save your table within ```ds3_key.db```:

```py
sql_merge.to_sql("sql_merge", con = connection)
```

In [14]:
sql_merge.to_sql("sql_merge", con = connection)

## Q3: Write a query that finds the average check per state. Then, for the most expensive state, find the most expensive restaurant. Is that also the highest rated restaurant for that state? Output all the columns for this question. 

In [15]:
sql_avg = pd.read_sql('''SELECT AVG(avg_check) as total_avg_checks, state
                        
                        FROM  sql_merge
                        GROUP BY state
                        ORDER BY total_avg_checks DESC ''', connection)
sql_avg

,total_avg_checks,state
0,106.000000,Mass.
1,97.454545,Nev.
2,83.000000,Ore.
3,82.500000,Ind.
4,82.444444,Ill.
5,81.500000,Ga.
6,76.761905,N.Y.
7,75.000000,N.C.
8,72.666667,Fla.
9,69.000000,Fla.


In [16]:
sql_fla = pd.read_sql('''SELECT *
                        
                        FROM  sql_merge
                        WHERE state = "Fla."
                        ORDER BY avg_check''', connection)
sql_fla

,index,rank,restaurant,sales,avg_check,city,state,meals_served,rating
0,1,2,The Boathouse Orlando,35218364,43,Orlando,Fla.,820819.0,4.0
1,53,59,The Rustic Inn,15700000,43,Ft. Lauderdale,Fla.,371000.0,4.0
2,23,27,Paddlefish,19829500,48,Orlando,Fla.,417500.0,4.0
3,34,39,Makoto,18216906,54,Bal Harbour,Fla.,256830.0,4.5
4,29,33,Shooters Waterfront,18601433,57,Ft. Lauderdale,Fla.,409752.0,4.0
5,60,66,Swan,15000000,78,Miami,Fla.,225000.0,3.5
6,15,19,Komodo,22500000,98,Miami,Fla.,265000.0,4.0
7,47,53,Rusty Pelican,16981741,98,Miami,Fla.,182000.0,3.5
8,10,12,Prime 112,23800000,135,Miami Beach,Fla.,206000.0,3.5


In [17]:
sql_most_exp_fla = pd.read_sql('''SELECT *
                        
                        FROM  sql_merge
                        WHERE state = "Fla."
                        ORDER BY avg_check desc
                        LIMIT 1''', connection)
sql_most_exp_fla

,index,rank,restaurant,sales,avg_check,city,state,meals_served,rating
0,10,12,Prime 112,23800000,135,Miami Beach,Fla.,206000.0,3.5


In [19]:
sql_most_rat_fla = pd.read_sql('''SELECT *
                        
                        FROM  sql_merge
                        WHERE state = "Fla."
                        ORDER BY rating desc
                        LIMIT 1''', connection)
sql_most_rat_fla

,index,rank,restaurant,sales,avg_check,city,state,meals_served,rating
0,34,39,Makoto,18216906,54,Bal Harbour,Fla.,256830.0,4.5


## Q4: Write your own query! Use the data given and the commands to find useful insights/something interesting in the data. Each group will share their findings!

In [18]:
...

Ellipsis